# OpenSearch Hybrid 검색을 통한 RAG
> 이 노트북은  SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

### 선수조건
- 이 노트북은 이전 노트북인 "02_KR_RAG_OpenSearch_Claude.ipynb" 이 완료 되었다고 가정 합니다.
    - 오픈서치 인텍스 관련 정보를 참조 합니다.

### Methods and Resources for Hybrid search with Re-ranking 
- Score Normalization
    - [MinMax based](https://towardsdatascience.com/text-search-vs-vector-search-better-together-3bd48eb6132a)
    - [z-socre based](https://towardsdatascience.com/hybrid-search-2-0-the-pursuit-of-better-search-ce44d6f20c08)
- Reciprocal Rank Fusion (RRF)
    - [Paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf)
    - [Description](https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d)
- [LangChain API for Ensemble Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)

### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.



In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

# 1. Bedrock Client 생성

In [7]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2)

In [4]:
from langchain.llms.bedrock import Bedrock

In [5]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={'max_tokens_to_sample': 512}
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f8ea820da50>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512})

## Embedding 모델 선택

In [6]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [ ]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)    

# 3. LangChain OpenSearch VectorStore 생성 
## 선수 조건
- 이전 노트북 02_1_KR_RAG_OpenSearch_Keyword.ipynb 또는 02_1_KR_RAG_OpenSearch_Semantic.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.

## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 OpenSearch ID/PW 를 입력을 해주세요.

In [ ]:
os.environ["OpenSearch_UserName"] = "Type ID" 
os.environ["OpenSearch_UserPassword"] = "Type PW" 

rag_user_name = os.environ["OpenSearch_UserName"]
rag_user_password = os.environ["OpenSearch_UserPassword"]

In [ ]:
opensearch_domain_endpoint = "https://search-genai-demo-domain-36etfsphifulgs7ckovsavchca.us-east-1.es.amazonaws.com"
http_auth = (rag_user_name, rag_user_password) # Master username, Master password

index_name = "genai-demo-index-v1-with-tokenizer"

## LangChain OpenSearch VectorStore 생성

In [ ]:
from langchain.vectorstores import OpenSearchVectorSearch

vectro_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

## OpenSearch Client 생성

In [ ]:
from utils.rag import create_aws_opensearch_client, check_if_index_exists, delete_index

In [ ]:
aws_client = create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 은전한잎 형태소 분석기 (seunjeon_tokenizer) 사용하기 in Opensearch
- 형태소 분석기에 대한 자세한 사항은 02_1_KR_RAG_OpenSearch_Keyword.ipynb 참고

### 인덱스 확인 (tokenization 확인)

In [ ]:
from pprint import pprint

In [ ]:
index_info = aws_client.indices.get(index=index_name)
pprint (index_info)

# 4. 오픈 서치에 "유사 서치" 검색
- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.



- similarity_search_with_score API 정보
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)

In [ ]:
import copy
from langchain.schema import Document
from langchain import PromptTemplate
from operator import itemgetter

## (1) OpenSearch Vector 검색

### 프로프트 템플릿 생성

In [ ]:
from utils.rag import create_bool_filter, run_RetrievalQA, show_context_used
from langchain.prompts import PromptTemplate

### [TIP] Prompt의 instruction의 경우 한글보다 **영어**로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [ ]:
# prompt_template = """
# \n\nHuman: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 응답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

# {context}

# Question: {question}
# \n\nAssistant:"""

prompt_template = """
\n\nHuman: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

\n\nAssistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


### 필터 생성

In [ ]:
filter01 = "인터넷뱅킹"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

# query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
query = "타기관OTP 등록 방법 알려주세요"

boolean_filter = create_bool_filter(filter01, filter02)
pprint (boolean_filter)

### LangChain RetrievalQA 를 통해 실행

In [ ]:
from termcolor import colored

In [ ]:
result = run_RetrievalQA(
    query,
    boolean_filter,
    llm_text,
    vectro_db,
    PROMPT,
    verbose=False,
    is_filter=True,
    k=5
)

print("##################################")
print("query: ", query)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(result['result'])

print (colored("\n\n### Contexts ###", "green"))
show_context_used(result['source_documents'])

## (2) OpenSearch Keyword 검색
- OpenSearch 에 아래와 같은 Query 를 실행하여 결과 받는 것을 구현합니다.

![keyword_filter.png](img/keyword_filter.png)

In [ ]:
from utils.rag import get_similiar_docs_with_keyword_score
from langchain.chains.question_answering import load_qa_chain

In [ ]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

In [ ]:
search_keyword_result = get_similiar_docs_with_keyword_score(
    query,
    index_name,
    aws_client,
    is_filter=True,
    filter01=filter01,
    filter02=filter02,
    k=10 # return up to k
)

answer = chain.run(
    input_documents=search_keyword_result,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)

### 키워드 검색 결과 (search_keyword_result)
bm25 score는 max_value로 normalization 되어 있음 (score range 0 - 1)

In [ ]:
search_keyword_result

## (3) OpenSearch Hybrid 검색

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) "Vector 서치" 하여 스코어를 얻은 후에 표준화를 하여 스코어를 구함. 
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (2) Keyword 서치도 동일하게 함.
- (3) 위의 두 개의 결과의 스코어를 소팅해서, Top K 를 구함.
    - 단 (1), (2) 의 동일한 항목일 경우는 (1) 스코어 + (2) 스코어 = 종합 스코어로 적용함. 
    - 즉 동일한 항목이 있으면 (1), (2)를 합산하기에 스코어가 더 높을 가능성이 많음.
    - 종합 스코어는 최대 2.0 이 될 수 있음. 이유는 (1) 에서 최대 스코어, (2)에서 최대 스코어 일 경우에 1.0 + 1.0 = 2.0 이 됨

스코어 표준화 예시

<pre>
Docs    Score	    Normalized-Score

Doc1: 	0.0083		1.0

Doc2: 	0.0074		0.8900

Doc3: 	0.0071		0.8585

Example: 0.0074 / 0.0083 = 0.8900
</pre>

In [ ]:
from utils.rag import get_similiar_docs
from utils.rag import interpolate_results, sort_score_search_results
from utils.rag import get_similiar_docs_with_keywords, get_similiar_docs

In [ ]:
def search_hybrid(query, vectro_db, aws_client, is_filter, boolean_filter, k=10):
    similar_docs_semantic = get_similiar_docs(
        query,
        vectro_db,
        is_filter=True,
        boolean_filter=boolean_filter,
        weight_decay_rate=0.1,  # 가중치 감소 비올: 에: 0, 0.1, 0.2  ..쵀대 0.9 (값이 작을수록 semantic 검색결과 많이 반영)
        k=k
    )            

    # print("##########################")
    # print("similar_docs_semantic: \n", similar_docs_semantic)
    # print("##########################")    
    similar_docs_keyword = get_similiar_docs_with_keywords(
        query,
        aws_client,
        index_name=index_name,
        weight_decay_rate=0,
        is_filter=True,
        filter01=filter01,
        filter02=filter02,
        k=k
    )
    # print("##########################")    
    # print("similar_docs_keyword: \n", similar_docs_keyword)    
    # print("##########################")    
    similar_docs = interpolate_results(similar_docs_semantic, similar_docs_keyword, k=k)
    # print("##########################")    
    # print("similar_docs: \n", similar_docs)
    # print("##########################")    
    # print("##--> Hybrid Search ")        
    
    answer = chain.run(input_documents=similar_docs, question=query)
    
    return answer

In [ ]:
%%time
# query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
query = "타기관OTP 등록 방법 알려주세요"
# question = "웹싸이트 계정몇 개 사용할 수 있나요?"

response = search_hybrid(query, vectro_db, aws_client, is_filter = True, boolean_filter = boolean_filter, k=3)

In [ ]:
print (f'question: {query}')
print (f'response: {response}')